In [12]:
#initial imports 
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Optional
import seaborn as sns
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Get the absolute path of the current file/notebook
# If using Jupyter, use Path.cwd(). If using a .py script, use Path(__file__).parent
curr_dir = Path.cwd()

# Calculate the project root (adjust '.parent' count as needed)
# If your notebook is in 'project/notebooks/', the root is 1 level up
project_root = curr_dir.parent.parent 

# Add project root to system path so Python can find 'utils'
sys.path.append(str(project_root))

print(f"Project Root added to path: {project_root}")

from utils.feature_engineer_df import build_features    

Project Root added to path: d:\Programming\ai_ds_bootcamp\ds-ml-project_kickstarters


In [3]:
build_features(
    input_path=Path.cwd().resolve().parents[1] / "data" / "cleaned" / "kickstarter_cleaned.csv",
    output_path=Path.cwd().resolve().parents[1] / "data" / "feature" / "kickstarter_featured.csv",
    raw_path=Path.cwd().resolve().parents[1] / "data" / "raw" / "ks-projects-201801.csv",
    logger=logger
)

# Load Files as DataFrames
BASE_DIR = Path.cwd().resolve().parents[1]
data_file = BASE_DIR / "data" / "feature" / "kickstarter_featured.csv"

filepath = Path(data_file)

df = pd.read_csv(filepath, encoding='latin-1', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/surya/Documents/neue_fische/11_project/ds-ml-project_kickstarters/data/feature/kickstarter_featured.csv'

In [4]:
df.head()

NameError: name 'df' is not defined

In [5]:
df.columns

NameError: name 'df' is not defined

### Creating our feature engineering dataset 

In [6]:
#list of columns to "hard drop" from feature engineering dataframe
columns_to_drop = ['id', #irrelevant
                   'main_category', #substituted in a satisfactory way
                   'deadline', 'launched', #created new categories 
                   'backers', 'usd_pledged_real', 'usd_pledged_bins', 'backers_per_pledged', 'backers_pledged_bins', 'pledged_per_category' #everything to do with "future information"
                   'launched_year', 'deadline_year', #info about the past and not seasonal
                   ]

In [7]:
#to remember: These would need to be soft-dropped later 
columns_to_softdrop = ['country', #to play around with
                       'duration_days', #because the bins are not ideal
                       'launched_month', 'deadline_month', #because we have season but might want to look closer
                       ]

In [8]:
dfc = df.drop(columns=columns_to_drop)
dfc.head()

NameError: name 'df' is not defined

#### Prepare categorical features for machine learning algorithms

We want to get dummies for all categoricals, that is 
* season_launched, season_deadline
* main_category_grouped
* continent

Open question: Scale or do dummies? 
* category_goal_percentile (as it's ordinal)
* duration bins (ordinal as well)

Scale:  
* usd_goal_real
* goal_per_category

Get dummies:

#question: how do we later know the legend? 

In [ ]:
#season launched, dropping first as it's multicollinear
df_sl = pd.get_dummies(df['launch_season'], prefix = 'sl_', drop_first=True, dtype=int)

#season deadline, dropping first
df_sd = pd.get_dummies(df['deadline_season'], prefix = 'sd_', drop_first=True, dtype=int)

#main category_grouped, dropping first
df_cat = pd.get_dummies(df['main_category_grouped'], prefix = 'cat_', drop_first=True, dtype=int)

#continent, dropping first
df_co = pd.get_dummies(df['continent'], prefix = 'co_', drop_first=True, dtype=int)

#put everything back together again: 
dff = pd.concat([df, df_sl, df_sd, df_cat, df_co], axis=1)

In [ ]:
#to remember: These would need to be soft-dropped later 
columns_to_softdrop = ['country', #to play around with
                       'duration_days', #because the bins are not ideal
                       'launched_month', 'deadline_month', #because we have season but might want to look closer
                       'launch_season', 'deadline_season', 'main_category_grouped', 'continent' #after creating dummies, get rid of these!
                       ]

Scale: